In [14]:
import pandas as pd

In [15]:
df = pd.read_pickle('Complete_NST_Text.pkl')

In [16]:
print df[:5]


Empty DataFrame
Columns: [Title, Date, Community (city,town, ward), LGA, State, Total Deaths, Boko Haram (P), State Actor (P), Sectarian Actor (excluding BH) (P), Other Armed Actor (P), Kidnapper (P), Robber (P), Other (P), Election-related Actor (P), Cameroon State Actor (P), Boko Haram (V), State Actor (V), Sectarian Actor (V), Other Armed Actor (V), Political Actor (V), Kidnapper (V), Kidnapee (V), Robber (V), Journalist (V), Civilian (V), Election-related Actor (V), Cameroon State Actor, Bomb, Gun, Machete, Suicide Bombing, Other Weapon, TK, Drinking Establishment, Goverment Building, Church, Mosque, Bank, School, Other Location, Notes, Sources 1, Sources 2, Sources 3, Latitude, Longitude, full place name, country, text]
Index: []

[0 rows x 49 columns]


In [3]:
nst_text = df[df['text']!='download failed']

In [6]:
len(nst_text[nst_text['State']!='Borno'])

0

In [7]:
nst_text['text'] = nst_text['text'].replace(r'\n', ' ', regex=True)

In [12]:
state_list = list(nst_text['State'].groupby(nst_text['State']).count().index)

In [13]:
state_dict = {}
for idx, state in enumerate(state_list):
    state_dict[state] = idx
    
from pprint import pprint
pprint(state_dict)

{}


In [10]:
nst_text = nst_text.dropna(subset=['State'])

In [63]:
nst_text['state_code'] = nst_text['State'].apply(lambda x: state_dict[x])

In [17]:
from sklearn.model_selection import train_test_split

In [65]:
X_train, X_test, y_train, y_test = train_test_split(nst_text['text'], nst_text['state_code'])

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [66]:
vect = CountVectorizer().fit(X_train)

In [67]:
X_train_vectorized = vect.transform(X_train)

In [25]:
X_train_vectorized

<2577x22534 sparse matrix of type '<class 'numpy.int64'>'
	with 405299 stored elements in Compressed Sparse Row format>

In [26]:
from sklearn.naive_bayes import GaussianNB

In [76]:
model = LogisticRegression(solver='newton-cg', multi_class='multinomial')
model.fit(X_train_vectorized, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [69]:
from sklearn.metrics import roc_auc_score

In [77]:
predictions = model.predict(vect.transform(X_test))

In [79]:
predictions

array([ 9, 28, 35, ..., 33, 23, 24])

In [82]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [83]:
accuracy_score(y_test, predictions)

0.7837078651685393

In [84]:
from sklearn.metrics import classification_report

In [85]:
classification_report(y_test, predictions)

/Users/aschharwood/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/aschharwood/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


'             precision    recall  f1-score   support\n\n          0       0.86      0.67      0.75         9\n          1       0.62      0.67      0.65        30\n          2       1.00      0.33      0.50        12\n          3       0.92      0.65      0.76        17\n          4       0.87      0.62      0.72        21\n          5       0.98      0.80      0.88        51\n          6       0.80      0.85      0.82        46\n          7       0.76      0.97      0.85       270\n          8       0.81      0.81      0.81        21\n          9       0.79      0.84      0.81        44\n         10       1.00      0.38      0.55        16\n         11       0.89      0.74      0.81        23\n         12       0.80      0.67      0.73        12\n         13       0.75      0.82      0.78        11\n         14       0.79      0.61      0.69        18\n         15       0.83      0.38      0.53        13\n         16       0.81      0.76      0.79        17\n         17       0.00   

In [86]:
precision_score(y_test, predictions, average='micro')

0.7837078651685393

In [87]:
precision_score(y_test, predictions, average='macro')

/Users/aschharwood/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.75925608064404537

In [88]:
f1_score(y_test, predictions, average='micro')

0.7837078651685393